In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# create a vocabulary


In [ ]:
import nltk
#nltk.download()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords


In [ ]:
#functions
# load doc into memory
def load_doc(filename):
# open the file as read only
  file = open(filename, 'r')
# read all text
  text = file.read()
# close the file
  file.close()
  return text

# turn a doc into clean tokens
def clean_doc(doc):
# split into tokens by white space
  tokens = doc.split()
# prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
  tokens = [re_punc.sub('', w) for w in tokens]
# remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
  stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
  tokens = [word for word in tokens if len(word) > 1]
  return tokens

# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
# load doc
  doc = load_doc(filename)

# clean doc
  tokens = clean_doc(doc)
# update counts
  vocab.update(tokens)

# load all docs in a directory
def process_docs(directory, vocab):
# walk through all files in the folder
  for filename in listdir(directory):
# skip any reviews in the test set
    if filename.startswith('cv9'):
      continue
# create the full path of the file to open
    path = directory + '/' + filename
# add doc to vocab
    add_doc_to_vocab(path, vocab)

def save_list(lines, filename):
# convert lines to a single blob of text
  data = '\n'.join(lines)
# open file
  file = open(filename, 'w')
# write text
  file.write(data)
# close file
  file.close()



In [ ]:
# define vocab
vocab = Counter()
# add all docs to vocab
#txt_sentoken = '/content/drive/MyDrive/txt_sentoken'
process_docs('/content/drive/MyDrive/txt_sentoken/pos', vocab)
process_docs('/content/drive/MyDrive/txt_sentoken/neg', vocab)
# print the size of the vocab
print(len(vocab))

44276


In [ ]:
#vocab


In [ ]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))
# save tokens to a vocabulary file
save_list(tokens, '/content/drive/MyDrive/txt_sentoken/vocab.txt')


25767


# Train a CNN


In [ ]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


In [ ]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
# split into tokens by white space
  tokens = doc.split()
# prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
  tokens = [re_punc.sub('', w) for w in tokens]
# filter out tokens not in vocab
  tokens = [w for w in tokens if w in vocab]
  tokens = ' '.join(tokens)
  return tokens


# load all docs in a directory
def process_docs(directory, vocab, is_train):
  documents = list()
# walk through all files in the folder
  for filename in listdir(directory):
# skip any reviews in the test set
    if is_train and filename.startswith('cv9'):
      continue
    if not is_train and not filename.startswith('cv9'):
      continue
# create the full path of the file to open
    path = directory + '/' + filename
# load the doc
    doc = load_doc(path)
# clean doc
    tokens = clean_doc(doc, vocab)
# add to list
    documents.append(tokens)
  return documents

# load and clean a dataset
def load_clean_dataset(vocab, is_train):
# load documents
  neg = process_docs('/content/drive/MyDrive/txt_sentoken/neg', vocab, is_train)
  pos = process_docs('/content/drive/MyDrive/txt_sentoken/pos', vocab, is_train)
  docs = neg + pos
# prepare labels
  labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
  return docs, labels

# fit a tokenizer
def create_tokenizer(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
# integer encode
  encoded = tokenizer.texts_to_sequences(docs)
# pad sequences
  padded = pad_sequences(encoded, maxlen=max_length, padding='post')
  return padded


In [ ]:
# define the model
def define_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 100, input_length=max_length))
  model.add(Conv1D(filters=32, kernel_size=12, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))
# compile network
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# summarize defined model
  model.summary()
  plot_model(model, to_file='model.png', show_shapes=True)

  return model


In [ ]:
# load the vocabulary
vocab_filename = '/content/drive/MyDrive/txt_sentoken/vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())

train_docs, ytrain = load_clean_dataset(vocab, True)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1

print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
# define model
model = define_model(vocab_size, max_length)
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# save the model
#model.save('model.h5')


Vocabulary size: 25768
Maximum length: 1317
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1317, 100)         2576800   
                                                                 
 conv1d_1 (Conv1D)           (None, 1306, 32)          38432     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 653, 32)          0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 20896)             0         
                                                                 
 dense_3 (Dense)             (None, 10)                208970    
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
          

array([  30, 2806,  325, ...,    0,    0,    0], dtype=int32)

In [ ]:
# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, max_length, model):
# clean review
  line = clean_doc(review, vocab)
# encode and pad review
  padded = encode_docs(tokenizer, max_length, [line])
# predict sentiment
  yhat = model.predict(padded, verbose=0)
# retrieve predicted percentage and label
  percent_pos = yhat[0,0]
  if round(percent_pos) == 0:
    return (1-percent_pos), 'NEGATIVE'
  return percent_pos, 'POSITIVE'


In [ ]:
test_docs, ytest = load_clean_dataset(vocab, False)

Xtest = encode_docs(tokenizer, max_length, test_docs)
_, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %.2f' % (acc*100))

Test Accuracy: 85.00


In [ ]:
text = 'Everyone will enjoy this film. I love it, recommended!'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))
# test negative text
text = 'averge . you can watch all by yourself.'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))

Review: [Everyone will enjoy this film. I love it, recommended!]
Sentiment: NEGATIVE (51.207%)
Review: [averge . you can watch all by yourself.]
Sentiment: NEGATIVE (51.860%)


# N-gram CNN model


In [ ]:
# turn a doc into clean tokens
def clean_doc(doc):
# split into tokens by white space
  tokens = doc.split()
# prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
  tokens = [re_punc.sub('', w) for w in tokens]
# filter out tokens not in vocab
  # remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
  stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
  tokens = [word for word in tokens if len(word) > 1]
  tokens = ' '.join(tokens)

  return tokens


# load all docs in a directory
def process_docs(directory, is_train):
  documents = list()
# walk through all files in the folder
  for filename in listdir(directory):
# skip any reviews in the test set
    if is_train and filename.startswith('cv9'):
      continue
    if not is_train and not filename.startswith('cv9'):
      continue
# create the full path of the file to open
    path = directory + '/' + filename
# load the doc
    doc = load_doc(path)
# clean doc
    tokens = clean_doc(doc)
# add to list
    documents.append(tokens)
  return documents

# load and clean a dataset
def load_clean_dataset(is_train):
# load documents
  neg = process_docs('/content/drive/MyDrive/txt_sentoken/neg',  is_train)
  pos = process_docs('/content/drive/MyDrive/txt_sentoken/pos',  is_train)
  docs = neg + pos
# prepare labels
  labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
  return docs, labels

# fit a tokenizer
def create_tokenizer(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
# integer encode
  encoded = tokenizer.texts_to_sequences(docs)
# pad sequences
  padded = pad_sequences(encoded, maxlen=max_length, padding='post')
  return padded


In [ ]:
from pickle import dump
def save_dataset(dataset, filename):
  dump(dataset, open(filename, 'wb'))
  print('Saved: %s' % filename)
# load and clean all reviews
train_docs, ytrain = load_clean_dataset(is_train=True)
test_docs, ytest = load_clean_dataset(is_train = False)
# save training datasets
save_dataset([train_docs, ytrain], 'train.pkl')
save_dataset([test_docs, ytest], 'test.pkl')


Saved: train.pkl
Saved: test.pkl


In [ ]:
from pickle import load
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import concatenate


In [ ]:

# load a clean dataset
def load_dataset(filename):
  return load(open(filename, 'rb'))
# fit a tokenizer
def create_tokenizer(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer
# calculate the maximum document length
def max_length(lines):
  return max([len(s.split()) for s in lines])
# encode a list of lines
def encode_text(tokenizer, lines, length):
# integer encode
  encoded = tokenizer.texts_to_sequences(lines)
# pad encoded sequences
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [ ]:
# define the model
def define_model(length, vocab_size):
# channel 1
  inputs1 = Input(shape=(length,))
  embedding1 = Embedding(vocab_size, 100)(inputs1)
  conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
  drop1 = Dropout(0.5)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  flat1 = Flatten()(pool1)
# channel 2
  inputs2 = Input(shape=(length,))
  embedding2 = Embedding(vocab_size, 100)(inputs2)
  conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
  drop2 = Dropout(0.5)(conv2)
  pool2 = MaxPooling1D(pool_size=2)(drop2)
  flat2 = Flatten()(pool2)
# channel 3
  inputs3 = Input(shape=(length,))
  embedding3 = Embedding(vocab_size, 100)(inputs3)
  conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
  drop3 = Dropout(0.5)(conv3)
  pool3 = MaxPooling1D(pool_size=2)(drop3)
  flat3 = Flatten()(pool3)
# merge
  merged = concatenate([flat1, flat2, flat3])
# interpretation
  dense1 = Dense(10, activation='relu')(merged)
  outputs = Dense(1, activation='sigmoid')(dense1)
  model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
# compile
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# summarize
  model.summary()
  plot_model(model, show_shapes=True, to_file='model.png')
  return model


In [ ]:
# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
print('Max document length: %d' % length)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
# define model
model = define_model(length, vocab_size)
# fit model
#model.fit([trainX,trainX,trainX], trainLabels, epochs=7, batch_size=16)
# save the model
#model.save('model.h5')

Max document length: 1380
Vocabulary size: 44277
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 1380)]       0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 1380)]       0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 1380)]       0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, 1380, 100)    4427700     ['input_7[0][0]']                
                                           

In [ ]:
# load training dataset
testLines, testLabels = load_dataset('test.pkl')
# create tokenizer

testX = encode_text(tokenizer, testLines, length)

_, acc = model.evaluate([testX,testX,testX], testLabels, verbose=0)
print('Test Accuracy: %.2f' % (acc*100))


Test Accuracy: 49.50


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
module_url = 'https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1'
embed_size = 128
trainable = False
hub_layer = hub.KerasLayer(module_url,input_shape=[],output_shape=[embed_size],dtype = tf.string,trainable = trainable)

# Character based CNN (LM)(cahracter based neural language modeling
)


In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras_preprocessing.sequence import pad_sequences


In [ ]:

#functions that we are gonna use in our later code

def load_doc(filename):
  file = open(filename,'r')

  text = file.read()
  file.close()

  return text

def save_doc(lines,filename):
  data = '\n'.join(lines)
  file = open(filename,'w')
  file.write(data)
  file.close()



def build_model(X):
  model = Sequential()
  model.add(LSTM(75,input_shape= (X.shape[1],X.shape[2])))
  model.add(Dense(vocab_size,activation = 'softmax'))

  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])
  model.summary()

  return model


In [ ]:
text = load_doc('/content/New Text Document.txt')
#print(text)

In [ ]:
tokens = text.split()
raw_text = ' '.join(tokens)
print(raw_text)

তবুও আমি ভাল থাকি , আকাশ থাকে যেমন, মেঘ ডাকে অজগুবি কে ডাকে অমনে


In [ ]:
length = 10
sequences = []
for i in range(length,len(raw_text)):
  seq = raw_text[i-length:i+1]
  sequences.append(seq)


In [ ]:
outfile = 'saved_text'
save_doc(sequences,outfile)

In [ ]:
#encode input sequence

inp_file = '/content/saved_text'
data = load_doc(inp_file)
lines = data.split('\n')



In [ ]:
chars = sorted(list(set(data)))
mapping = dict((c,i) for i, c in enumerate(chars))



In [ ]:
encoded_sequence = []

for line in lines:
  en_sq = [mapping[char] for char in line]
  encoded_sequence.append(en_sq)

print('length of vacab is ' + str(len(mapping)) )





length of vacab is 24


In [ ]:
vocab_size = len(mapping)


In [ ]:
import numpy as np
encoded_sequence = np.array(encoded_sequence)

In [ ]:
X = encoded_sequence[:,:-1]
y = encoded_sequence[:,-1]

In [ ]:


tr_x = np.array([to_categorical(x, num_classes = vocab_size ) for x in X])
tr_y = to_categorical(y,num_classes = vocab_size)

In [ ]:
model = build_model(tr_x)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                30000     
                                                                 
 dense (Dense)               (None, 24)                1824      
                                                                 
Total params: 31,824
Trainable params: 31,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(tr_x,tr_y,epochs = 150)

Epoch 1/150
2/2 [==============================] - 2s 19ms/step - loss: 3.1555 - accuracy: 0.0556
Epoch 2/150
2/2 [==============================] - 0s 14ms/step - loss: 3.1374 - accuracy: 0.0926
Epoch 3/150
2/2 [==============================] - 0s 16ms/step - loss: 3.1220 - accuracy: 0.1852
Epoch 4/150
2/2 [==============================] - 0s 13ms/step - loss: 3.1021 - accuracy: 0.2222
Epoch 5/150
2/2 [==============================] - 0s 16ms/step - loss: 3.0848 - accuracy: 0.2037
Epoch 6/150
2/2 [==============================] - 0s 13ms/step - loss: 3.0650 - accuracy: 0.1852
Epoch 7/150
2/2 [==============================] - 0s 13ms/step - loss: 3.0386 - accuracy: 0.1852
Epoch 8/150
2/2 [==============================] - 0s 12ms/step - loss: 3.0093 - accuracy: 0.1852
Epoch 9/150
2/2 [==============================] - 0s 17ms/step - loss: 2.9727 - accuracy: 0.2037
Epoch 10/150
2/2 [==============================] - 0s 12ms/step - loss: 2.9290 - accuracy: 0.2037
Epoch 11/150
2/2 [=

In [ ]:
def generate_seq(model , mapping , seq_length , seed, n_chars):
  import numpy as np
  in_text = seed
  for _ in range(n_chars):
# encode the characters as integers
    encoded = [mapping[char] for char in in_text]
# truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
# one hot encode
    encoded = to_categorical(encoded, num_classes=len(mapping))
    #encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
# predict character
    yhat = model.predict(encoded)

    classes_x=np.argmax(yhat,axis=1)

    out_char = ''

    for char , index in mapping.items():
      if index == classes_x:
        out_char = char
        break

    in_text += out_char

  return in_text


In [ ]:
print(generate_seq(model, mapping, 10, 'যেমন', 3))
# test mid-line
#print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
#print(generate_seq(model, mapping, 10, 'hello worl', 20))

1/1 [==============================] - 0s 22ms/step
যেমনে  


In [ ]:
print(generate_seq(model, mapping, 10, 'sing a son', 20))


1/1 [==============================] - 0s 22ms/step
sing a song of sixpence, A poc


In [ ]:
print(generate_seq(model, mapping, 10, 'king was i', 20))


1/1 [==============================] - 0s 32ms/step
king was in his counting house


In [ ]:
print(generate_seq(model, mapping, 10, 'i lo ', 20))

1/1 [==============================] - 0s 23ms/step
i lo ke pe. Foffo  theepi
